# Training the UNET

## Import

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras.models import Model
from keras.layers import Input, Dense
import keras.losses as losses


### 전처리

In [2]:
import cv2

n_train_B = 400
# n_train_A = 10
n_test = 20

train_green = []

for i in range (0,n_train_B,3):
    filename = "..\capture\draw-" + str(i) + ".bmp"

    try:
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR->RGB
        image = cv2.resize(image,(256,256), interpolation=cv2.INTER_AREA)
    except Exception as e:
        print(str(e))
    image = np.array(image)
    train_green.append(image)
    
train_green = np.array(train_green, dtype="float32")
train_green /= 255.0

In [3]:
train_purple = []

for i in range (3,n_train_B+3,3):
    filename = "../capture/draw-" + str(i) + ".bmp"
    try:
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        image = cv2.resize(image,(256,256), interpolation=cv2.INTER_AREA)
    except Exception as e:
        print(str(e)+str(i))
    image = np.array(image)
    train_purple.append(image)
    
train_purple = np.array(train_purple, dtype="float32")
train_purple /= 255.0

### 모델 생성

In [4]:
input_g = tf.keras.Input(shape=(256,256,3))
        
conv0 = layers.Conv2D(16, (3,3), activation='relu', padding = 'same')(input_g)
mp0 = layers.MaxPooling2D((2,2))(conv0)
        
conv1 = layers.Conv2D(32, (3,3), activation='relu', padding = 'same')(mp0)
mp1 = layers.MaxPooling2D((2,2))(conv1)

conv2 = layers.Conv2D(64, (3,3), activation='relu', padding = 'same')(mp1)
mp2 = layers.MaxPooling2D((2,2))(conv2)
        
conv3 = layers.Conv2D(128, (3,3), activation='relu', padding = 'same')(mp2)
mp3 = layers.MaxPooling2D((2,2))(conv3)
        
conv4 = layers.Conv2D(256, (3,3), activation='relu', padding = 'same')(mp3)
mp4 = layers.MaxPooling2D((2,2))(conv4)
        
output_e = layers.Conv2D(512, (3,3), activation='relu', padding = 'same')(mp4)

convt1 = layers.Conv2DTranspose(256, (3,3), activation='relu', padding='same')(output_e) #512아니고 256
upsamp1 = layers.UpSampling2D((2,2))(convt1)
skipcon1 = layers.Concatenate(axis=3)([conv4, upsamp1])
conv6 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(skipcon1)

convt2 = layers.Conv2DTranspose(128, (3,3), activation='relu', padding='same')(conv6)
upsamp2 = layers.UpSampling2D((2,2))(convt2)
skipcon2 = layers.Concatenate(axis=3)([conv3, upsamp2])
conv7 = layers.Conv2D(128, (3,3), activation = 'relu', padding='same')(skipcon2)

convt3 = layers.Conv2DTranspose(64, (3,3), activation='relu', padding='same')(conv7)
upsamp3 = layers.UpSampling2D((2,2))(convt3)
skipcon3 = layers.Concatenate(axis=3)([conv2, upsamp3])
conv8 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(skipcon3)

convt4 = layers.Conv2DTranspose(32, (3,3), activation='relu', padding='same')(conv8)
upsamp4 = layers.UpSampling2D((2,2))(convt4)
skipcon4 = layers.Concatenate(axis=3)([conv1, upsamp4])
conv9 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(skipcon4)
        
convt5 = layers.Conv2DTranspose(16, (3,3), activation='relu', padding='same')(conv9)
upsamp5 = layers.UpSampling2D((2,2))(convt5)
skipcon5 = layers.Concatenate(axis=3)([conv0, upsamp5])
conv10 = layers.Conv2D(16, (3,3), activation='relu', padding='same')(skipcon5)

output_p = layers.Conv2DTranspose(3, (3,3), activation='relu', padding='same')(conv10)

### 학습

In [8]:
model = Model(inputs=input_g, outputs=output_p)
model.compile(optimizer='adam', loss=losses.MeanSquaredError())
model.fit(train_green, train_purple, validation_split=0.2, epochs=100, verbose=2)

Epoch 1/100
4/4 - 8s - loss: 0.0092 - val_loss: 0.0068 - 8s/epoch - 2s/step
Epoch 2/100
4/4 - 7s - loss: 0.0051 - val_loss: 0.0057 - 7s/epoch - 2s/step
Epoch 3/100
4/4 - 7s - loss: 0.0045 - val_loss: 0.0051 - 7s/epoch - 2s/step
Epoch 4/100
4/4 - 6s - loss: 0.0037 - val_loss: 0.0039 - 6s/epoch - 2s/step
Epoch 5/100
4/4 - 6s - loss: 0.0030 - val_loss: 0.0034 - 6s/epoch - 2s/step
Epoch 6/100
4/4 - 6s - loss: 0.0027 - val_loss: 0.0029 - 6s/epoch - 1s/step
Epoch 7/100
4/4 - 6s - loss: 0.0023 - val_loss: 0.0026 - 6s/epoch - 2s/step
Epoch 8/100
4/4 - 6s - loss: 0.0021 - val_loss: 0.0024 - 6s/epoch - 2s/step
Epoch 9/100
4/4 - 7s - loss: 0.0018 - val_loss: 0.0022 - 7s/epoch - 2s/step
Epoch 10/100
4/4 - 6s - loss: 0.0017 - val_loss: 0.0021 - 6s/epoch - 2s/step
Epoch 11/100
4/4 - 6s - loss: 0.0015 - val_loss: 0.0020 - 6s/epoch - 2s/step
Epoch 12/100
4/4 - 6s - loss: 0.0015 - val_loss: 0.0019 - 6s/epoch - 2s/step
Epoch 13/100
4/4 - 6s - loss: 0.0014 - val_loss: 0.0018 - 6s/epoch - 1s/step
Epoch 14

### 테스트

In [5]:
test_green = []

for i in range (n_train_B+2,n_train_B+n_test+1,3):
    filename = "E:/Project_Ch00cy/CGVR/TextureSynthesis/deep-textures-main/testingTextures/capture/draw-" + str(i) + ".bmp"

    try:
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256,256), interpolation=cv2.INTER_AREA)
    except Exception as e:
        print(str(e)+str(i))
    image = np.array(image)
    test_green.append(image)

test_green = np.array(test_green, dtype="float32")
test_green /= 255.0

In [6]:
# # 5. 모델 평가하기
# loss_and_metrics = model.evaluate(test_green, train_purple, batch_size=32)
# print('')
# print('loss_and_metrics : ' + str(loss_and_metrics))

# 6. 모델 저장하기
from keras.models import load_model
model.save('mnist_mlp_model.h5')

NameError: name 'model' is not defined

#### 있는 모델 사용하기

In [7]:
# 2. 모델 불러오기
from keras.models import load_model
model = load_model('mnist_mlp_model.h5')

# 3. 모델 사용하기
yhat = model.predict(test_green)


1/1 [==============================] - 0s 466ms/step


In [2]:
test_green.shape

NameError: name 'test_green' is not defined

In [1]:
for i in range(len(test_green)):
    plt.imshow(test_green[i])
    plt.show()

NameError: name 'test_green' is not defined

### 이미지 결과 확인

In [11]:
def generate_images(test_input, test_output):
    plt.figure(figsize=(10, 10))
    display_list = [test_input, test_output]
    title = ['Input Image', 'Predicted Image']
    
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

In [10]:
test_prediction = model(test_green)
len(test_prediction)

7

In [12]:
test_prediction

<tf.Tensor: shape=(7, 256, 256, 3), dtype=float32, numpy=
array([[[[0.03268217, 0.01520426, 0.02719468],
         [0.03932293, 0.02598581, 0.04171288],
         [0.04635206, 0.03315447, 0.05396435],
         ...,
         [0.06663042, 0.06237805, 0.06767373],
         [0.05374959, 0.05282442, 0.0531797 ],
         [0.03240607, 0.04216038, 0.0290646 ]],

        [[0.02853904, 0.01148754, 0.02229679],
         [0.04650098, 0.03219425, 0.03904272],
         [0.0553173 , 0.04721168, 0.05370128],
         ...,
         [0.08141539, 0.07918234, 0.08350819],
         [0.06476713, 0.07103483, 0.0744247 ],
         [0.03587205, 0.04971919, 0.04209115]],

        [[0.02142425, 0.00812062, 0.01571565],
         [0.03938732, 0.0308584 , 0.03077213],
         [0.0502337 , 0.04377071, 0.04836468],
         ...,
         [0.06712113, 0.07136527, 0.0704251 ],
         [0.05528385, 0.06771279, 0.06298744],
         [0.03206268, 0.04188643, 0.03912257]],

        ...,

        [[0.        , 0.        , 

In [1]:
plt.imshow(test_prediction[3])
plt.show()

NameError: name 'plt' is not defined

In [ ]:
generate_images(test_prediction[1],test_prediction[2])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 

In [ ]:
plt.imshow((test_prediction[1] * 255).astype(np.uint8))

AttributeError: EagerTensor object has no attribute 'astype'. 
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()
      

In [ ]:
test_prediction = model(test_green)

i = 0
for image in test_prediction:
    generate_images(test_green[i], image)
    i += 1

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 